In [1]:
import PySimpleGUI as sg      
from GasDynamicsSim import ShockSim
from DriverInsert import InsertOpt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os

C:\Users\Jackie Zheng\PycharmProjects\StanShock\stanShock.py:394: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "getCp_python" failed type inference due to: No implementation of function Function(<built-in function zeros>) found for signature:
 
 >>> zeros(int64, dtype=Function(<class 'int'>))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload of function 'zeros': File: numba\core\typing\npydecl.py: Line 507.
    With argument(s): '(int64, dtype=Function(<class 'int'>))':
   No match.

During: resolving callee type: Function(<built-in function zeros>)
During: typing of call at C:\Users\Jackie Zheng\PycharmProjects\StanShock\stanShock.py (417)


File "..\stanShock.py", line 417:
def getCp_python(T,Y,TTable,a,b):
    <source elided>
    #determine the indices
    indices = np.zeros(nX,dtype=int)
    ^

  def getCp_python(T,Y,TTable,a,b):
C:\Users\Jackie Zheng\PycharmProjects\StanShock\stanShock.py

In [2]:
# set up a function for including plots in windows
matplotlib.use("TkAgg")
    
def draw_figure(canvas, figure):
    figure_canvas_agg = FigureCanvasTkAgg(figure, canvas)
    figure_canvas_agg.draw()
    figure_canvas_agg.get_tk_widget().pack(side="top", fill="both", expand=1)
    return figure_canvas_agg

# define function to open new windows
def open_window(layout, name = "Multi-stage Backfill Options", click = "Apply"):
    #window = sg.Window("Second Window", layout, modal=True)
    window = sg.Window(name, layout)
    #choice = None
    while True:
        event, values = window.read()
        if event == click or event == sg.WIN_CLOSED:
            return window, event, values
            break
        
    window.close()
    
# define function to save simulation results
def save_simulation(sim):
    layout = [[
        sg.InputText(key='Where to Save', default_text='file location', enable_events=True),
        sg.InputText(size=(12, 1), key='Simulation Number', default_text='simulation #', enable_events=True),
        sg.InputText(key='File Location', do_not_clear=False, enable_events=True, visible=False),
        sg.FolderBrowse(initial_folder='/tmp'),
        sg.Button('Save File')
    ]]
    window = sg.Window('Save Simulation Results', layout)

    while True:
        event, values = window.Read()
        #print("event:", event, "values: ",values)
        if event is None or event == 'Exit':
            break
        elif event == 'File Location':
            file_loc = values['File Location']
            if file_loc:
                window['Where to Save'].update(value=file_loc)
        elif event == 'Save File':
            if not file_loc:
                sg.popup('Save location not specified!')
            elif file_loc:
                try:
                    cwd = os.getcwd()
                    os.chdir(file_loc)
                    folder_name = '\Sim_'+ values['Simulation Number']
                    os.mkdir(file_loc + folder_name)
                    os.chdir(file_loc + folder_name)
                    #sg.popup(file_loc + folder_name)
                    np.savetxt('TMatrix.csv', sim.TMatrix, delimiter=',')
                    np.savetxt('pMatrix.csv', sim.pMatrix, delimiter=',')
                    np.savetxt('timeXT.csv', sim.timeXT, delimiter=',')
                    np.savetxt('positionXT.csv', sim.positionXT, delimiter=',')
                    if not any(opt.xInsert == None):
                        np.savetxt('xInsert.csv', sim.xInsert, delimiter=',')
                        np.savetxt('DInnerInsert.csv', sim.DInnerInsert, delimiter=',')
                    os.chdir(cwd)
                    sg.popup('Files saved successfully!')
                    break
                except:
                    window.close()
                    sg.popup('Something went wrong...Try again!')
                    break
    window.close()
    
sg.theme('LightBlue')    
# define mechanism choices
MechChoices = ('O2Argon.xml', 'Propane.xml', 'frosh_v1.1.cti')
# define input column items
LeftColumn = [    
          [sg.Text('Mechanism'),sg.Listbox(MechChoices, size=(15, len(MechChoices)), key='mechanism', enable_events=True)],
          [sg.Frame(layout=[ # block for defining driver gases
              [sg.Text('Driver gas 1:'),sg.Input(size=(15, 1), default_text='N2', key='derg1'), sg.Text('mole fraction:'),sg.Input(size=(15, 1), default_text='1', key='Xderg1')],  
              [sg.Text('Driver gas 2:'),sg.Input(size=(15, 1), key='derg2'), sg.Text('mole fraction:'),sg.Input(size=(15, 1), key='Xderg2')]
                           ], title='Driver Gas')],
          [sg.Frame(layout=[ # block for defining backfill, if backfill option is chosen
              [sg.Checkbox('Backfill', default=False, key= 'backfill'), sg.Button('Multi-stage')],
              [sg.Text('Backfill gas :'),sg.Input(size=(15, 1), key='bfg'), sg.Text('Backfill length(m):'),sg.Input(size=(15, 1), key='LBackfill')]
                           ], title='Backfill Options')],
          [sg.Frame(layout=[ # block to define test gases   
              [sg.Text('Driven gas 1:'),sg.Input(size=(15, 1), default_text='AR', key='dveng1'), sg.Text('mole fraction:'),sg.Input(size=(15, 1), default_text='1', key='Xdveng1')],
              [sg.Text('Driven gas 2:'),sg.Input(size=(15, 1), key='dveng2'), sg.Text('mole fraction:'),sg.Input(size=(15, 1), key='Xdveng2')],
              [sg.Text('Driven gas 3:'),sg.Input(size=(15, 1), key='dveng3'), sg.Text('mole fraction:'),sg.Input(size=(15, 1), key='Xdveng3')]
                           ], title='Test Gas')],
          [sg.Frame(layout=[ # block to define buffer gases, if CRV option is chosen
              [sg.Checkbox('CRV', default=False, key= 'CRV')],
              [sg.Text('Buffer gas :'),sg.Input(size=(15, 1), key='buffg'), sg.Text('Buffer fraction:'),sg.Input(size=(15, 1), key='BufferFraction')]
                           ], title='CRV Options')],
          [sg.Frame(layout=[ # block to define shock conditions     
              [sg.Text('P4(psi):   '),sg.Input(size=(15, 1), default_text='30', key='p4')],  
              [sg.Text('P1(torr):  '),sg.Input(size=(15, 1), default_text='50', key='p1')],
              [sg.Text('T1(C):     '),sg.Input(size=(15, 1), default_text='21', key='T1')]
                           ], title='Shock Conditions'),
          sg.Frame(layout=[ # block to define simulation parameters
              [sg.Text('Number of grid divisions:   '),sg.Input(size=(15, 1), default_text='500', key='nXFine')],
              [sg.Text('Simulation end time(ms):         '),sg.Input(size=(15, 1), default_text='25', key='tFinal')],
              [sg.Checkbox('Boundary Layer Term', default=True, key= 'BL'),sg.Checkbox('Reacting', default=False, key= 'react'),sg.Checkbox('Diffusion', default=False, key= 'diffuse')]
                           ], title='Simulation Parameters')],
          [sg.Frame(layout=[ # block to define driver insert optimization parameters
              [sg.Text('Number of coarse grid divisions:   '),sg.Input(size=(15, 1), default_text='150', key='nXCoarse')],
              [sg.Text('Test time to be optimized(ms):'),sg.Input(size=(15, 1), default_text='2', key='tTest')]
                           ], title='Driver Insert Optimization Parameters'),
           sg.Frame(layout=[ # block to define simulation parameters
              [sg.Text('Alpha:   '),sg.Input(size=(5, 1), default_text='1', key='alpha')],
              [sg.Text('Beta:     '),sg.Input(size=(5, 1), default_text='1', key='beta')],
              [sg.Text('D:         '),sg.Input(size=(5, 1), default_text='1', key='D_mul')]
                           ], title='Compressibility Factor')],
          [sg.Frame(layout=[ # block to define shock tube geometries
              [sg.Text('Driver length(m):'),sg.Input(size=(7, 1), default_text='3.72', key='LDriver'), sg.Text('Driver diameter(cm):'),sg.Input(size=(7, 1),default_text='11.53', key='DDriver')],
              [sg.Text('Driven length(m):'),sg.Input(size=(7, 1), default_text='9.73',key='LDriven'), sg.Text('Driven diameter(cm):'),sg.Input(size=(7, 1), default_text='11.53',key='DDriven')]
                           ], title='Shock Tube Geometry'), 
           sg.Frame(layout=[# block to set up driver inserts
              [sg.Checkbox('Driver Insert', default=False, key= 'insert')],
              [sg.Button('Specify Inserts')] 
                           ], title='Driver Insert')],
          [sg.Button('Run Simulation'), sg.Button('Run Driver Insert Optimization'), sg.Button('Save Simulation Results'), sg.Exit()]]      

# a right column for displaying simulation results
RightColumn = [
               [sg.Text('X-t Diagram:'), sg.Text('                                                                                              End Wall Pressure:')],
               [sg.Canvas(key="XTDiagram"), sg.Canvas(key="EndWallPressure")],
               [sg.Text('Driver Section Configuration:')],
               [sg.Canvas(key="DriverConfiguration")],
               [sg.Text('Track simulation progress here:')],
               [sg.Output(size=(125,10), key='-OUTPUT-')],
              ]

# combine two cloumns for a final layout
layout = [
    [
        sg.Column(LeftColumn),
        sg.VSeperator(),
        sg.Column(RightColumn),
    ]
]

# set up main window
window = sg.Window('StanShock Shock Tube Simulation', layout)      

# set up backfill sub-window
layout_multistage = [    
              [sg.Frame(layout=[ # block for defining backfill stages
                  [sg.Text('Backfill gases stage 1:'),sg.Input(size=(15, 1), default_text='N2', key='mbf1a'), sg.Input(size=(15, 1), default_text='HE', key='mbf1b'), sg.Text('mole fractions:'),sg.Input(size=(15, 1),  default_text='0.5',key='Xmbf1a'), sg.Input(size=(15, 1),  default_text='0.5',key='Xmbf1b')],  
                  [sg.Text('Backfill gases stage 2:'),sg.Input(size=(15, 1), default_text='N2',key='mbf2a'), sg.Input(size=(15, 1), key='mbf2b'), sg.Text('mole fractions:'),sg.Input(size=(15, 1), default_text='1',key='Xmbf2a'), sg.Input(size=(15, 1), key='Xmbf2b')],
                  [sg.Text('Backfill gases stage 3:'),sg.Input(size=(15, 1), key='mbf3a'), sg.Input(size=(15, 1), key='mbf3b'), sg.Text('mole fractions:'),sg.Input(size=(15, 1),key='Xmbf3a'), sg.Input(size=(15, 1), key='Xmbf3b')],
                  [sg.Text('Backfill gases stage 4:'),sg.Input(size=(15, 1), key='mbf4a'), sg.Input(size=(15, 1), key='mbf4b'), sg.Text('mole fractions:'),sg.Input(size=(15, 1), key='Xmbf4a'), sg.Input(size=(15, 1), key='Xmbf4b')],
                  [sg.Text('Backfill gases stage 5:'),sg.Input(size=(15, 1), key='mbf5a'), sg.Input(size=(15, 1), key='mbf5b'), sg.Text('mole fractions:'),sg.Input(size=(15, 1), key='Xmbf5a'), sg.Input(size=(15, 1), key='Xmbf5b')],
                  [sg.Text('Backfill gases stage 6:'),sg.Input(size=(15, 1), key='mbf6a'), sg.Input(size=(15, 1), key='mbf6b'), sg.Text('mole fractions:'),sg.Input(size=(15, 1), key='Xmbf6a'), sg.Input(size=(15, 1), key='Xmbf6b')],
                               ], title='Backfill Stages')],
              [sg.Frame(layout=[ # block to define backfill geometries
                  [sg.Text('Stage 1 length(m):'),sg.Input(size=(15, 1), default_text='12', key='Lbf1')],
                  [sg.Text('Stage 2 length(m):'),sg.Input(size=(15, 1), default_text='5',key='Lbf2')],
                  [sg.Text('Stage 3 length(m):'),sg.Input(size=(15, 1), key='Lbf3')],
                  [sg.Text('Stage 4 length(m):'),sg.Input(size=(15, 1), key='Lbf4')],
                  [sg.Text('Stage 5 length(m):'),sg.Input(size=(15, 1), key='Lbf5')],
                  [sg.Text('Stage 6 length(m):'),sg.Input(size=(15, 1), key='Lbf6')],
                               ], title='Backfill Stage Length')],
              [sg.Button('Apply')]
                    ]

# set up driver insert sub-window
layout_insert = [
              [sg.Frame(layout=[ # block for defining backfill stages
                  [sg.Text('N: 4.5",   M: 4.0",   L+: 3.7",   L: 3.6",    K: 3.3"')],  
                  [sg.Text('J: 3.1",    I: 2.9",    H: 2.6",    G: 2.55",   F+:2.5"')],
                  [sg.Text('F: 2.4",   E: 2.3",   D: 2.1",    D-: 2.0",   C+:1.8"')],
                  [sg.Text('C: 1.7",   B: 1.5",   A: 1.4",    O: 1.1",    Rod:0.5"')]
                               ], title='Reference for Insert Diameter')],
              [sg.Frame(layout=[ # block for defining backfill stages
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='F+', key='Ins1'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='10', key='LIns1')],  
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='F', key='Ins2'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='22', key='LIns2')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='E', key='Ins3'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='22', key='LIns3')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='D', key='Ins4'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='8', key='LIns4')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='D-', key='Ins5'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='10', key='LIns5')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='C+', key='Ins6'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='10', key='LIns6')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='C', key='Ins7'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='3', key='LIns7')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='B', key='Ins8'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='8', key='LIns8')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='A', key='Ins9'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='7', key='LIns9')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='O', key='Ins10'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='6', key='LIns10')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), default_text='Rod', key='Ins11'), sg.Text('Insert length:'), sg.Input(size=(7, 1), default_text='11', key='LIns11')],
                  [sg.Text('Insert class:'), sg.Input(size=(7, 1), key='Ins12'), sg.Text('Insert length:'), sg.Input(size=(7, 1), key='LIns12')],
                               ], title='Specify Insert Profile to Simulate')],
              [sg.Button('Apply')]
                ]

# initialize event loop
event = []
values = {}
multiStage = False # default multi-stage to false
XBackfill = None
LBackfill = np.array([])
Insert = {} # initialize driver insert profile
SimulateInsert = False # default insert simulation to false
ssim = None
while True:                             # The Event Loop
    event, values = window.read() 
    
    # setup multi-stage backfill if the button is clicked
    
    # initialize variables
    X4 = ''
    X1 = ''
    
    if event == 'Multi-stage':
        try:
            if not values['backfill']:
                sg.popup('Enable backfill first!')
            elif values['backfill']:
                window_multistage, event_multistage, values_multistage = open_window(layout_multistage)
                if event_multistage == 'Apply':
                    multiStage = True
                    #print('Code is Running')
                    # set up gas in backfill stages to interface with cantera
                    stage1 = (values_multistage['mbf1a'], values_multistage['mbf1b'])
                    stage2 = (values_multistage['mbf2a'], values_multistage['mbf2b'])
                    stage3 = (values_multistage['mbf3a'], values_multistage['mbf3b'])
                    stage4 = (values_multistage['mbf4a'], values_multistage['mbf4b'])
                    stage5 = (values_multistage['mbf5a'], values_multistage['mbf5b'])
                    stage6 = (values_multistage['mbf6a'], values_multistage['mbf6b'])
                    allStages = (stage1, stage2, stage3, stage4, stage5, stage6)
                    # set up mole fraction of each gas
                    Xstage1 = (values_multistage['Xmbf1a'], values_multistage['Xmbf1b'])
                    Xstage2 = (values_multistage['Xmbf2a'], values_multistage['Xmbf2b'])
                    Xstage3 = (values_multistage['Xmbf3a'], values_multistage['Xmbf3b'])
                    Xstage4 = (values_multistage['Xmbf4a'], values_multistage['Xmbf4b'])
                    Xstage5 = (values_multistage['Xmbf5a'], values_multistage['Xmbf5b'])
                    Xstage6 = (values_multistage['Xmbf6a'], values_multistage['Xmbf6b'])
                    XallStages = (Xstage1, Xstage2, Xstage3, Xstage4, Xstage5, Xstage6)
                    #set up backfill length for each stage
                    LallStages = (values_multistage['Lbf1'], 
                                  values_multistage['Lbf2'], 
                                  values_multistage['Lbf3'], 
                                  values_multistage['Lbf4'], 
                                  values_multistage['Lbf5'], 
                                  values_multistage['Lbf6'])
                    # initialize the final cantera gas object definition
                    which_stage = 0
                    while not allStages[which_stage][0] == '' and which_stage <6:
                        Xcurrent_stage = ''
                        for i in range(0, len(allStages[which_stage])):
                            if not allStages[which_stage][i] == '' and Xcurrent_stage == '':  # if the very first gas is being added
                                Xcurrent_stage = allStages[which_stage][i]+':'+XallStages[which_stage][i]
                            elif not allStages[which_stage][i] == '' and not Xcurrent_stage == '':  # if not the first gas being added
                                Xcurrent_stage = Xcurrent_stage + ','+ allStages[which_stage][i]+':'+XallStages[which_stage][i]
                        # add the gas definition from the current stage to the collection XBackfill
                        if XBackfill == None:
                            XBackfill = (Xcurrent_stage, )
                        else:
                            XBackfill = XBackfill + (Xcurrent_stage, )
                        # add backfill length to LBackfill array
                        LBackfill = np.append(LBackfill, float(LallStages[which_stage]))
                        # advance the stage step until the last stage is read
                        if which_stage<5:
                            which_stage = which_stage+1
                        else:
                            break
                    print('Backfill stages are:')
                    print(XBackfill)
                    print('Backfill length are:')
                    print(LBackfill)
                    window_multistage.close()
        except:
            window_multistage.close()
            window.close()
            raise Exception('Something went wrong...Try again!')
            break
    
    if event == 'Specify Inserts':
        if not values['insert']:
                sg.popup('Enable driver inserts first!')
        elif values['insert']:
            #sg.popup('Insert menu still under construction!')
            window_insert, event_insert, values_insert = open_window(layout_insert)
            if event_insert == "Apply":
                SimulateInsert = True
                InsSections = (values_insert['Ins1'], 
                               values_insert['Ins2'],
                               values_insert['Ins3'],
                               values_insert['Ins4'],
                               values_insert['Ins5'],
                               values_insert['Ins6'],
                               values_insert['Ins7'],
                               values_insert['Ins8'],
                               values_insert['Ins9'],
                               values_insert['Ins10'],
                               values_insert['Ins11'],
                               values_insert['Ins12'])
                LInsSections = (values_insert['LIns1'], 
                               values_insert['LIns2'],
                               values_insert['LIns3'],
                               values_insert['LIns4'],
                               values_insert['LIns5'],
                               values_insert['LIns6'],
                               values_insert['LIns7'],
                               values_insert['LIns8'],
                               values_insert['LIns9'],
                               values_insert['LIns10'],
                               values_insert['LIns11'],
                               values_insert['LIns12'])
                section = 0
                while not InsSections[section] == '' and section <11:
                    try:
                        Insert[InsSections[section]] = float(LInsSections[section])
                        section = section +1
                    except:
                        window_insert.close()
                        raise Exception('Something went wrong...Try again!')
                window_insert.close()    
                print('Driver insert profile is:')
                print(Insert)
                
            
    if event == 'Run Simulation':
        try:
            # write driver gas
            driverGas = (values['derg1'], values['derg2'])
            XdriverGas = (values['Xderg1'], values['Xderg2'])
            # loop to obtain all defined driver gas composition
            for i in range(0, len(driverGas)):
                if not driverGas[i] == ''and X4 == '':
                    X4 = driverGas[i]+':'+XdriverGas[i]
                elif not driverGas[i] == ''and not X4 == '':
                    X4 = X4 + ',' +driverGas[i]+':'+XdriverGas[i]
            # write test gas
            testGas = (values['dveng1'], values['dveng2'], values['dveng3'])
            XtestGas = (values['Xdveng1'], values['Xdveng2'], values['Xdveng3'])
            # loop to obtain all defined test gas composition
            for i in range(0, len(testGas)):
                if not testGas[i] == '' and X1 == '':
                    X1 = testGas[i]+':'+XtestGas[i]
                elif not testGas[i] == '' and not X1 == '':
                    X1 = X1+ ',' +testGas[i]+':'+XtestGas[i]
            
            # read some inputs for single stage backfill
            if not multiStage and values['backfill']:  # if multistage backfill is not selected, set backfill length to simple stage value
                XBackfill = values['bfg']+':1'
                LBackfill = [float(values['LBackfill'])]
            
            # read the inputs for CRV 
            XBuffer = values['buffg'] +':1'
            
            # interface with StanShock wrapper function
            Mixture = {
                'mechanism': values['mechanism'][0], 
                'X1': X1, 
                'X4': X4,
                'XBuffer':XBuffer,
                'XBackfill':(XBackfill)
                      }
                
            Thermal = {
                'p4':float(values['p4'])*6894.76, # psi to Pa 
                'p1':float(values['p1'])*133.322, # torr to Pa
                'T1':float(values['T1'])+273, # Celcius to Kelvin
            }

            Sim = {
                'nXFine':int(values['nXFine']), #grid definition
                'tFinal':float(values['tFinal'])*(1e-3), # simulation time
                'Backfill': values['backfill'],
                'CRV': values['CRV'],
                'alpha':float(values['alpha']), # boundary layer correction term for shear stress. 1 for incompressible case
                'beta':float(values['beta']), # boundary layer correction term for heat transfer. 1 for incompressible case
                'D_mul':float(values['D_mul']), # boundary layer correction term for effective diameter. 1 for incompressible case
                'BoundaryLayer': values['BL'],
                'Reacting': values['react'],
                'Diffusion': values['diffuse']
            }

            if values['BufferFraction'] == '':
                values['BufferFraction'] = 0
            if values['LBackfill'] == '':
                values['LBackfill'] = 0

            Geometry = {
                'LDriver':float(values['LDriver']),# driver section length
                'LDriven':float(values['LDriven']),# driven section length
                'DDriver':float(values['DDriver'])*(1e-2), # driver section diameter
                'DDriven':float(values['DDriven'])*(1e-2), # driven section diameter
                'LBackfill':LBackfill,
                'BufferFraction':float(values['BufferFraction'])
            }

                
            # run simulation here
            if not SimulateInsert:
                print('Running simulation without driver insert...')
                ssim = ShockSim(Mixture, Thermal, Sim, Geometry, saveData = False, GUI=True)
                ssim.XT_Diagram()
                # get all the plots
                figT = ssim.figT
                figP = ssim.figP
                figEndWallT = ssim.figEndWallT
                figEndWallP = ssim.figEndWallP
                figDriver = ssim.figDriver
                ssim.xInsert = None
                ssim.DInnerInsert = None
            elif SimulateInsert:
                print('Running simulation with driver insert...')
                opt = InsertOpt(Mixture, Thermal, Sim, Geometry, saveData = False, GUI=True)
                opt.SimulateInsert(discrete = True, Insert = Insert)
                # get all the plots
                #figT = opt.figT
                figP = opt.figP
                #figEndWallT = opt.figEndWallT
                figEndWallP = opt.figEndWallP
                figDriver = opt.figDriver
                
            draw_figure(window["XTDiagram"].TKCanvas, figP)
            draw_figure(window["EndWallPressure"].TKCanvas, figEndWallP)
            draw_figure(window["DriverConfiguration"].TKCanvas, figDriver)
        except:
            window.close()
            raise Exception('Something went wrong...Try again!')
            break
    
    # define driver optimization routine when the button is hit
    if event == 'Run Driver Insert Optimization':
        if values['insert']:
            sg.popup('Uncheck the insert checkbox first, since we are still trying to determine the insert profile.')
        elif not values['BL']:
            sg.popup('Enable boundary layer term first!')
        else:
            try:
                # write driver gas
                driverGas = (values['derg1'], values['derg2'])
                XdriverGas = (values['Xderg1'], values['Xderg2'])
                # loop to obtain all defined driver gas composition
                for i in range(0, len(driverGas)):
                    if not driverGas[i] == ''and X4 == '':
                        X4 = driverGas[i]+':'+XdriverGas[i]
                    elif not driverGas[i] == ''and not X4 == '':
                        X4 = X4 + ',' +driverGas[i]+':'+XdriverGas[i]
                # write test gas
                testGas = (values['dveng1'], values['dveng2'], values['dveng3'])
                XtestGas = (values['Xdveng1'], values['Xdveng2'], values['Xdveng3'])
                # loop to obtain all defined test gas composition
                for i in range(0, len(testGas)):
                    if not testGas[i] == '' and X1 == '':
                        X1 = testGas[i]+':'+XtestGas[i]
                    elif not testGas[i] == '' and not X1 == '':
                        X1 = X1+ ',' +testGas[i]+':'+XtestGas[i]

                # read some inputs for single stage backfill
                if not multiStage and values['backfill']:  # if multistage backfill is not selected, set backfill length to simple stage value
                    XBackfill = values['bfg']+':1'
                    LBackfill = [float(values['LBackfill'])]

                # read the inputs for CRV 
                XBuffer = values['buffg'] +':1'

                # interface with StanShock wrapper function
                Mixture = {
                    'mechanism': values['mechanism'][0], 
                    'X1': X1, 
                    'X4': X4,
                    'XBuffer':XBuffer,
                    'XBackfill':(XBackfill)
                          }

                Thermal = {
                    'p4':float(values['p4'])*6894.76, # psi to Pa 
                    'p1':float(values['p1'])*133.322, # torr to Pa
                    'T1':float(values['T1'])+273, # Celcius to Kelvin
                }

                Sim = {
                    'nXCoarse':int(values['nXCoarse']),
                    'nXFine':int(values['nXFine']), #grid definition
                    'xStep':10,
                    'tFinal':float(values['tFinal'])*(1e-3), # simulation time
                    'Backfill': values['backfill'],
                    'CRV': values['CRV'],
                    'alpha':float(values['alpha']), # boundary layer correction term for shear stress. 1 for incompressible case
                    'beta':float(values['beta']), # boundary layer correction term for heat transfer. 1 for incompressible case
                    'D_mul':float(values['D_mul']), # boundary layer correction term for effective diameter. 1 for incompressible case
                    'BoundaryLayer': values['BL'],
                    'Reacting': values['react'],
                    'Diffusion': values['diffuse'],
                    'tTest':float(values['tTest'])*(1e-3),
                    'tradeoff':0
                }

                if values['BufferFraction'] == '':
                    values['BufferFraction'] = 0
                if values['LBackfill'] == '':
                    values['LBackfill'] = 0

                Geometry = {
                    'LDriver':float(values['LDriver']),# driver section length
                    'LDriven':float(values['LDriven']),# driven section length
                    'DDriver':float(values['DDriver'])*(1e-2), # driver section diameter
                    'DDriven':float(values['DDriven'])*(1e-2), # driven section diameter
                    'LBackfill':LBackfill,
                    'BufferFraction':float(values['BufferFraction'])
                }
                print('Running driver insert optimization...')
                opt = InsertOpt(Mixture, Thermal, Sim, Geometry, saveData = False, GUI=True)
                opt.GetInsert()
                # get all the plots
                #figT = opt.figT
                figP = opt.figP
                #figEndWallT = opt.figEndWallT
                figEndWallP = opt.figEndWallP
                figDriver = opt.figDriver

                draw_figure(window["XTDiagram"].TKCanvas, figP)
                draw_figure(window["EndWallPressure"].TKCanvas, figEndWallP)
                draw_figure(window["DriverConfiguration"].TKCanvas, figDriver)
            except:
                window.close()
                raise Exception('Something went wrong...Try again!')
                break
    
    # save data option is defined here:
    if event == 'Save Simulation Results':
        if not ssim == None:
            save_simulation(ssim)
        elif not opt == None:
            save_simulation(opt)
        else:
            sg.popup('Run the simulation first!')
                
    # close the program by hitting exit or closing the window    
    if event == sg.WIN_CLOSED or event == 'Exit':
        break      

window.close()